In [1]:
from openai import OpenAI

# Point to the local server
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")
client.models.list()

SyncPage[Model](data=[Model(id='gemma-3-27b-it', created=None, object='model', owned_by='organization_owner'), Model(id='phi-3.5-vision-instruct', created=None, object='model', owned_by='organization_owner'), Model(id='text-embedding-nomic-embed-text-v1.5@q4_k_m', created=None, object='model', owned_by='organization_owner'), Model(id='llama-3.3-70b-instruct', created=None, object='model', owned_by='organization_owner'), Model(id='qwen2-7b-instruct', created=None, object='model', owned_by='organization_owner'), Model(id='meta-llama-3.1-70b-instruct', created=None, object='model', owned_by='organization_owner'), Model(id='mistral-7b-instruct-v0.3', created=None, object='model', owned_by='organization_owner'), Model(id='llama-3-8b-instruct-64k', created=None, object='model', owned_by='organization_owner'), Model(id='gemma-2-27b-it', created=None, object='model', owned_by='organization_owner'), Model(id='meta-llama-3.1-8b-instruct', created=None, object='model', owned_by='organization_owne

In [10]:
import os
import pandas as pd

# Load the CSV
df = pd.read_csv("representative_votes_clean_format.csv")

# Rename Unnamed: 0 → case_ID if it exists
if "Unnamed: 0" in df.columns:
    df.rename(columns={"Unnamed: 0": "case_ID"}, inplace=True)

# Define image folder
image_dir = "gsv/final_photo_dataset"

# Function to check if an image file exists (with any common extension)
def has_image(image_id):
    for ext in [".JPG", ".jpg", ".jpeg"]:
        if os.path.exists(os.path.join(image_dir, f"{image_id}{ext}")):
            return True
    return False

# Filter rows where both left and right images exist
df["left_exists"] = df["left"].astype(str).apply(has_image)
df["right_exists"] = df["right"].astype(str).apply(has_image)

# Keep only rows where both exist
df_cleaned = df[df["left_exists"] & df["right_exists"]].copy()

# Drop helper columns
df_cleaned.drop(columns=["left_exists", "right_exists"], inplace=True)

# Save result
df_cleaned.to_csv("representative_votes_clean_format_images_exist.csv", index=False)
print("✅ Saved to 'representative_votes_clean_format_images_exist.csv'")


✅ Saved to 'representative_votes_clean_format_images_exist.csv'


In [12]:
import os
from PIL import Image, ImageOps
import pandas as pd

# Load data
df = pd.read_csv("representative_votes_clean_format_images_exist.csv")

image_dir = "gsv/final_photo_dataset"
output_dir = "merged_images"
os.makedirs(output_dir, exist_ok=True)

separator_width = 100
separator_color = (255, 255, 255)

def get_image_path(base_name):
    for ext in [".JPG", ".jpg", ".jpeg"]:
        path = os.path.join(image_dir, f"{base_name}{ext}")
        if os.path.exists(path):
            return path
    return None

# Track metadata
matched_rows = []
missing_logs = []

merged_index = 0
row_idx = 0

while merged_index < 100 and row_idx < len(df):
    row = df.iloc[row_idx]
    row_idx += 1

    left_id = str(row["left"])
    right_id = str(row["right"])

    left_path = get_image_path(left_id)
    right_path = get_image_path(right_id)

    if not left_path or not right_path:
        if not left_path:
            missing_logs.append(f"Row {row_idx - 1}: Missing left image '{left_id}'")
        if not right_path:
            missing_logs.append(f"Row {row_idx - 1}: Missing right image '{right_id}'")
        continue  # Skip this row

    try:
        left_img = Image.open(left_path)
        right_img = Image.open(right_path)

        max_height = max(left_img.height, right_img.height)
        left_img = ImageOps.pad(left_img, (left_img.width, max_height))
        right_img = ImageOps.pad(right_img, (right_img.width, max_height))

        separator = Image.new("RGB", (separator_width, max_height), separator_color)

        merged = Image.new("RGB", (left_img.width + separator_width + right_img.width, max_height))
        merged.paste(left_img, (0, 0))
        merged.paste(separator, (left_img.width, 0))
        merged.paste(right_img, (left_img.width + separator_width, 0))

        filename = f"merged_{merged_index:03d}.jpg"
        merged.save(os.path.join(output_dir, filename))

        matched_rows.append({
            "merged_index": merged_index,
            "row_index": row["case_ID"],
            "left": left_id,
            "right": right_id,
            "study_question": row["study_question"],
            "choice": row["choice"],
            "place_name_left":row["place_name_left"],
            "place_name_right":row["place_name_right"],
            "left_vote":row["left_num"],
            "right_vote":row["right_num"]
        })

        merged_index += 1

    except Exception as e:
        missing_logs.append(f"Row {row_idx - 1}: ERROR during merge: {e}")

# Save metadata
pd.DataFrame(matched_rows).to_csv("merged_metadata.csv", index=False)

# Print missing log
print("✅ Merged images saved to 'merged_images/'")
print("✅ Metadata saved to 'merged_metadata.csv'")
print("⚠️ Missing or error rows:")
for log in missing_logs:
    print(log)


✅ Merged images saved to 'merged_images/'
✅ Metadata saved to 'merged_metadata.csv'
⚠️ Missing or error rows:


In [ ]:
import os
from PIL import Image, ImageOps
import pandas as pd

# Load data
df = pd.read_csv("representative_votes_clean_format_images_exist.csv")

image_dir = "gsv/final_photo_dataset"
output_dir = "merged_images"
os.makedirs(output_dir, exist_ok=True)

separator_width = 100
separator_color = (255, 255, 255)

def get_image_path(base_name):
    for ext in [".JPG", ".jpg", ".jpeg"]:
        path = os.path.join(image_dir, f"{base_name}{ext}")
        if os.path.exists(path):
            return path
    return None

# Track metadata
matched_rows = []
missing_logs = []

merged_index = 0
row_idx = 0

for merged_index, row in df.iterrows():
    row = df.iloc[row_idx]
    row_idx += 1

    left_id = str(row["left"])
    right_id = str(row["right"])

    left_path = get_image_path(left_id)
    right_path = get_image_path(right_id)

    if not left_path or not right_path:
        if not left_path:
            missing_logs.append(f"Row {row_idx - 1}: Missing left image '{left_id}'")
        if not right_path:
            missing_logs.append(f"Row {row_idx - 1}: Missing right image '{right_id}'")
        continue  # Skip this row

    try:
        left_img = Image.open(left_path)
        right_img = Image.open(right_path)

        max_height = max(left_img.height, right_img.height)
        left_img = ImageOps.pad(left_img, (left_img.width, max_height))
        right_img = ImageOps.pad(right_img, (right_img.width, max_height))

        separator = Image.new("RGB", (separator_width, max_height), separator_color)

        merged = Image.new("RGB", (left_img.width + separator_width + right_img.width, max_height))
        merged.paste(left_img, (0, 0))
        merged.paste(separator, (left_img.width, 0))
        merged.paste(right_img, (left_img.width + separator_width, 0))

        filename = f"merged_{merged_index:03d}.jpg"
        merged.save(os.path.join(output_dir, filename))

        matched_rows.append({
            "merged_index": merged_index,
            "row_index": row["case_ID"],
            "left": left_id,
            "right": right_id,
            "study_question": row["study_question"],
            "choice": row["choice"],
            "place_name_left":row["place_name_left"],
            "place_name_right":row["place_name_right"],
            "left_vote":row["left_num"],
            "right_vote":row["right_num"]
        })

        merged_index += 1

    except Exception as e:
        missing_logs.append(f"Row {row_idx - 1}: ERROR during merge: {e}")

# Save metadata
pd.DataFrame(matched_rows).to_csv("merged_metadata.csv", index=False)

# Print missing log
print("✅ Merged images saved to 'merged_images/'")
print("✅ Metadata saved to 'merged_metadata.csv'")
print("⚠️ Missing or error rows:")
for log in missing_logs:
    print(log)
